In [3]:
import numpy as np
def DFT_slow(x):
    """Compute the discrete Fourier Transform of the 1D array"""
    x = np.asarray(x, dtype=float)
    N = x.shape[0] # num col in x
    n = np.arange(N) # array of val 1:N
    k = n.reshape((N,1)) # transpose
    M = np.exp(-2j*np.pi*k*n/N)
    return np.dot(M,x)
    

In [19]:
x = np.random.random(1024)
np.allclose(DFT_slow(x), np.fft.fft(x))


True

In [16]:
%timeit DFT_slow(x)
%timeit np.fft.fft(x)

156 ms ± 8.13 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
26.6 µs ± 412 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [31]:
def FFT(x):
    """A recursive implementation of the 1D Cooley-Tukey FFT"""
    x = np.asarray(x,dtype=float)
    N = x.shape[0]
    
    if N % 2 > 0:
        raise ValueError("size of x must be a power of 2")
    elif N <= 32:
        return DFT_slow(x)
    else:
        X_even = FFT(x[::2])
        X_odd = FFT(x[1::2])
        factor = np.exp(-2j*np.pi*np.arange(N)/N)
        return np.concatenate([X_even + factor[:N//2]*X_odd,
                               X_even + factor[N//2:]*X_odd])
    

In [32]:
x = np.random.random(1024)
np.allclose(FFT(x), np.fft.fft(x))

True

In [37]:
def FFT_vectorized(x):
    """A vectorized, non-recursive version of the Cooley-Turkey FFT"""
    x = np.asarray(x, dtype=float)
    N = x.shape[0]
    
    if np.log2(N) % 1 > 0:
        raise ValueError("size of x must be a power of 2")
    
    # N_min here is equivalent to the stopping condition above,
    # and should be a power of 2
    N_min = min(N, 32)
    
    # Perform an O[N^2] DFT on all length-N_min subproblems at once
    n = np.arange(N_min)
    k = n[:,None]
    M = np.exp(-2j*np.pi*n*k/N_min)
    X = np.dot(M, x.reshape((N_min,-1))) # reshape here takes the transpose
    
    # build-up of each level of the recursive calculations all at once
    while X.shape[0] < N:
        X_even = X[:, :X.shape[1]//2]
        X_odd = X[:, X.shape[1]//2:]
        factor = np.exp(-1j*np.pi*np.arange(X.shape[0])
                        /X.shape[0])[:,None]
        X = np.vstack([X_even + factor * X_odd,
                       X_even - factor * X_odd])
        
    return X.ravel()

In [38]:
x = np.random.random(1024)
np.allclose(FFT_vectorized(x), np.fft.fft(x))

True

In [39]:
%timeit FFT(x)
%timeit FFT_vectorized(x)
%timeit np.fft.fft(x)

2.66 ms ± 94.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
339 µs ± 29.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
17.3 µs ± 637 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [ ]:
def two_comp_dec(x):
    for i in range(len(x)):
        if (x[i] & (1 << 15)):
            x[i] = x[i]-2**16
        print(x[i])

In [4]:
x = (2+4j, -5+35j)
print(np.fft.fft(x))

x = (100+70j, -200+35j)
print(np.fft.fft(x))

x = (1+5j, 2+6j, 3+7j, 4+8j)
print(np.fft.fft(x))

x = (3000-2000j, 23+0j, -4000-1500j, 9000+8j)
print(np.fft.fft(x))

x = (3000-2000j, 23+0j, -4000-1500j, 9000+8j, 1+5j, 2+6j, 3+7j, 4+8j)
np.fft.fft(x)

[-3.+39.j  7.-31.j]
[-100.+105.j  300. +35.j]
[10.+26.j -4. +0.j -2. -2.j  0. -4.j]
[  8023.-3492.j   6992.+8477.j -10023.-3508.j   7008.-9477.j]


array([12954.         -4809.j        ,  8844.79333436 -6644.24910262j,
        -987.65548008 +3067.20809122j, -6834.72596525+14332.9945241j ,
       -3396.        +12731.j        ,  4379.62333818 -6809.89827901j,
        9344.41442382-25054.69803878j,  6727.54732719-23828.08895181j,
        3304.         -2003.j        ,  1900.81199832+18806.04821566j,
        4117.65548008+19064.79190878j,  3432.33780983 +1991.93055743j,
        -366.        -13619.j        , -3505.22867087-13543.90083404j,
        -170.41442382 -4817.30196122j,  8254.84082824  -864.83612973j])

In [37]:
# Prints out test values for testing
x = [100+100j, 150+150j, 200+200j, 130+120j, 90+70j, 40+20j, -10-30j, -40-70j, \
     -80-110j, -140-160j, -100-120j, -70-80j, -30-30j, 10+5j, 40+25j, 80+45j]
x_real = np.real(x)
x_imag = np.imag(x)
x_real = x_real.astype(dtype=int)
x_imag = x_imag.astype(dtype=int)
x_real_bin = []
x_imag_bin = []
for i in range(16):
    x_real_bin.append(np.binary_repr(x_real[i], 16))
    x_imag_bin.append(np.binary_repr(x_imag[i], 16))
for i in range(16):
    print('input_real', end='')
    print(i, end=' ')
    print('<=', end=' 16\'b')
    print(x_real_bin[i], end=';\n')
    print('input_imag', end='')
    print(i, end=' ')
    print('<=', end=' 16\'b')
    print(x_imag_bin[i], end=';\n')
    

input_real0 <= 16'b0000000001100100;
input_imag0 <= 16'b0000000001100100;
input_real1 <= 16'b0000000010010110;
input_imag1 <= 16'b0000000010010110;
input_real2 <= 16'b0000000011001000;
input_imag2 <= 16'b0000000011001000;
input_real3 <= 16'b0000000010000010;
input_imag3 <= 16'b0000000001111000;
input_real4 <= 16'b0000000001011010;
input_imag4 <= 16'b0000000001000110;
input_real5 <= 16'b0000000000101000;
input_imag5 <= 16'b0000000000010100;
input_real6 <= 16'b1111111111110110;
input_imag6 <= 16'b1111111111100010;
input_real7 <= 16'b1111111111011000;
input_imag7 <= 16'b1111111110111010;
input_real8 <= 16'b1111111110110000;
input_imag8 <= 16'b1111111110010010;
input_real9 <= 16'b1111111101110100;
input_imag9 <= 16'b1111111101100000;
input_real10 <= 16'b1111111110011100;
input_imag10 <= 16'b1111111110001000;
input_real11 <= 16'b1111111110111010;
input_imag11 <= 16'b1111111110110000;
input_real12 <= 16'b1111111111100010;
input_imag12 <= 16'b1111111111100010;
input_real13 <= 16'b000000000000

In [49]:
print('Expected Results')
print(np.fft.fft(x))

y_real = [\
    0b0000000101101000,\
    0b0000000000110100,\
    0b1111111111010000,\
    0b1111111111010000,\
    0b0000000000011010\
]
y_imag = [\
    
]
print('\nActual Results - Real')
two_comp_dec(y_real)
print('Actual Results - Imaginary')
two_comp_dec(y_imag)

Expected Results
[ 370.         +135.j         1431.97483106 +364.28510797j
   23.78679656 -176.56854249j  117.61618122 -189.26132408j
  -50.           -5.j          -56.83178629  -59.15342397j
  -61.36038969 +104.85281374j  -30.48534663  +30.94802596j
   50.          +75.j           -2.2334902    -7.50841268j
   66.21320344  -63.43145751j  -77.82433402  -34.6221524j
  -50.          -85.j         -252.90955457  +62.37672868j
 -188.63961031  -64.85281374j  310.69349943+1512.93545053j]

Actual Results - Real
360
52
-48
-48
26
Actual Results - Imaginary


In [ ]:
x = [-3, -2, -1, 0, 1, 2, 3]
x = [0b1111001001110000, \
0b1110111011100000, \
0b0010000100011011,\
0b1100111110100101, \
0b1111001001001100,\
0b0010000010111010,\
0b1101101011111001,\
0b0000000101110001]
two_comp_dec(x)